In [108]:
import pandas as pd
import string


import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stopwords = stopwords.words('english')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /Users/damirs/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /Users/damirs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/damirs/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [91]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [92]:
test.head(3)

,id,review
0,21681,"Sweet, entertaining tale of a young 17 1/2 yea..."
1,10829,"This movie had an interesting cast, it mat not..."
2,37269,This film grabs you from the opening scenes an...


In [93]:
train.head(3)

,review,sentiment,id
0,I argued with myself whether to rent this or n...,negative,41449
1,This was one of the dullest movies I have seen...,negative,18376
2,"I didn't know what to expect from 'Ned Kelly',...",positive,31081


In [94]:
def remove_stopwords(text):
    s = []
    for words in text.split():
        if words not in stopwords:
            s.append(words)
    a = s[:]
    s.clear()
    return ' '.join(a)

def preprocess_text(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    cleaned_text = ' '.join(tokens)
    return cleaned_text

def stemming(data):
    stemmer = PorterStemmer()
    text = [stemmer.stem(word) for word in data]
    return data
    
def cleaning_review(data, column):
    data[column] = data[column].str.replace(r'<[^<>]*>', '', regex = True)
    data[column] = data[column].str.replace(r'https ? ://\s+|www\.\s+', '', regex = True)
    data[column] = data[column].str.lower()
    data[column] = data[column].str.replace('[{}]'.format(string.punctuation), '', regex = True)
    data[column] = data[column].apply(remove_stopwords)
    data[column] = data[column].apply(preprocess_text)
    data[column] = data[column].apply(lambda x: stemming(x))
    
#     data['words'] = data[column].apply(word_tokenize)
    return data


train = cleaning_review(train, 'review')
train['sentiment'] = train['sentiment']

test = cleaning_review(test, 'review')


# train['words'] = train['review'].apply(word_tokenize)

In [95]:
train.head(3)

,review,sentiment,id
0,argued whether rent im always afraid renting s...,negative,41449
1,one dullest movie seen time im late 40 watched...,negative,18376
2,didnt know expect ned kelly absolutely loved d...,positive,31081


In [96]:
test.head(3)

,id,review
0,21681,sweet entertaining tale young 17 12 year old b...
1,10829,movie interesting cast mat list cast actor fil...
2,37269,film grab opening scene never let go watch ind...


In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
x_train = vect.fit_transform(train['review'])

x_test = vect.transform(test['review'])

In [98]:
# x_train = train['review']
y_train = train['sentiment']
# x_test = test['review']
y_test = pd.read_csv('sample.csv')['sentiment']

In [99]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import warnings
warnings.filterwarnings('ignore')

In [101]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
logreg_pred = logreg.predict(x_test)

# Multinomia Nayve Bayes
multnb = MultinomialNB().fit(x_train, y_train)
multnb_pred = multnb.predict(x_test)

In [104]:
test['sentiment'] = logreg_pred
# test.drop(columns='review', inplace=True)
test.to_csv('predicted_sentiments.csv', index=False)

In [102]:
test['sentiment'] = multnb_pred
test.drop(columns='review', inplace=True)
test.to_csv('pred_nb.csv', index=False)

In [107]:
test

,id,sentiment
0,21681,positive
1,10829,positive
2,37269,positive
3,13397,positive
4,41784,positive
...,...,...
9995,45578,positive
9996,22733,negative
9997,39642,negative
9998,5386,negative
